In [3]:
# !pip install faiss
# !sudo apt-get install libopenblas-dev
# !sudo apt-get install libomp-dev

In [4]:
import pandas as pd

df = pd.read_csv('/content/City_data2.csv')
df1 = df[['city','beach','mountain','history','food','city_life','countryside','nightlife','couple_friendly','outdoor','spiritual','hot','cold','mild','snowy','desert','rainy','athletics']]
df1.head()

,city,beach,mountain,history,food,city_life,countryside,nightlife,couple_friendly,outdoor,spiritual,hot,cold,mild,snowy,desert,rainy,athletics
0,Paris,0,0,1,1,1,0,1,1,0,0,0,1,1,1,0.0,1,1
1,New york,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0.0,1,1
2,Rome,1,0,1,1,1,1,1,1,1,1,1,0,1,0,0.0,0,0
3,London,1,1,1,1,1,0,1,1,0,0,0,1,1,1,0.0,1,1
4,Tokyo,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0.0,0,1


In [5]:
df2 = df1.set_index('city').T.to_dict('list')
df2

{'Abu Dhabi': [1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 'Adelaide': [1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0],
 'Agra': [0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Amman': [0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 'Amsterdam': [0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Anchorage': [0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0],
 'Apia': [1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0],
 'Arequipa': [0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  

In [6]:
import numpy as np
data = dict()
names = []
vectors = []


for a,b in df2.items():
    names.append(a)
    vectors.append(b)

data['name'] = np.array(names, dtype=object)
data['vector'] = np.array(vectors, dtype=float)

In [16]:
import faiss
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
      self.index = faiss.IndexLSH(self.dimension, num_bits)
      self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors.reshape(-1,17).astype('float32'), k) 
        return [self.labels[i] for i in indices[0]]

In [17]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [20]:
import time
place_vector, place_name = data['vector'][list(data['name']).index("Taipei")], data['name'][list(data['name']).index("Taipei")]
start = time.time()
simlar_place_names = '\n* '.join(index.query(place_vector))
print(simlar_place_names)
end = time.time()
print('time taken '+ str(end - start))

Qatar
* Bali
* Kuala Lumpur
* Rome
* Hanoi
* Kingston
* Taipei
* Panama City
* Jerusalem
* Bangkok
time taken 0.0073626041412353516


In [ ]:
## Taipei itself goes to 5th index, so performing not that great